## Monolingual Norec experiments with bert
These were originally in the Experiments2 notebook. I have not run them since I moved them here, so maybe there is an undefined variable

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import pandas as pd
import pickle as pk
import os
import random
import json
import time, datetime
random.seed(64)
# import nltk
import re
# from nltk.tokenize.simple import SpaceTokenizer
from helpers import *
from keep_records import Keep_records

print(f"Cuda: {torch.cuda.is_available()}")

Cuda: True


In [3]:
datafolder = "norne/data/"
train_all = datafolder + "no_bokmaal-ud-train.bmes"
dev_all = datafolder + 'no_bokmaal-ud-dev.bmes'
train_4c = datafolder + "no_bokmaal-ud-train_4c.bmes"
dev_4c = datafolder + 'no_bokmaal-ud-dev_4c.bmes'

dataset_4c = dataset_w_tags(train_4c, dev_4c)
dataset_all = dataset_w_tags(train_all, dev_all)
datasets = [dataset_4c, dataset_all]
keeper = Keep_records()
for ds in datasets:
    print(ds[-1])

['B-MISC', 'B-PER', 'B-ORG', 'I-LOC', 'I-ORG', 'O', 'B-LOC', 'I-PER', 'I-MISC']


In [4]:
# Run bert multilingual with the settings from previous cell

# family = "bert"
family = "xlmroberta"
# transformersmodel = 'bert-base-multilingual-cased'
transformersmodel = 'xlm-roberta-base'
results = []

for train_path, dev_path, tags in datasets:
    model_args = NERArgs() # New args loading fall 2020
    model_args.train_batch_size = 16
    model_args.num_train_epochs = 8
    model_args.weight_decay = 0.001
    model_args.overwrite_output_dir = True
    model_args.silent = True
    model_args.save_steps = -1
    model_args.evaluate_during_training = True

    model = NERModel(family, transformersmodel, labels = tags, args=model_args)

    out_d = "outputs/"+transformersmodel+"_"+keeper.ts
    model.train_model(train_path, output_dir= out_d, eval_data=dev_path)
    print(transformersmodel, "Done training")

    result, model_outputs, predictions = model.eval_model(dev_path)

    #Record settings and results
    result["train"] = train_path
    result["dev_test"] = dev_path
    result["training_epochs"] = model_args.num_train_epochs
    result["transformer_model"] = transformersmodel
    keeper.keep(result)

df = keeper.get_df()



 results_1059.json


In [5]:
# Run all above
df

,eval_loss,precision,recall,f1_score,train,dev_test,training_epochs,transformer_model,timestamp
0,0.017232,0.919859,0.924763,0.922304,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,1,xlm-roberta-base,111005


In [6]:
keeper.to_csv(df.round(3))